In [1]:
import time
import sys
import os
from collections import OrderedDict
from importlib import reload
from pprint import pprint

import h5py
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import qcodes
qc = qcodes

In [3]:
from scipy import constants
from scipy.interpolate import interp1d
from scipy.optimize import fmin
from scipy.integrate import cumtrapz

In [4]:
from labtools.mplplots import init_nb_plotting
reload(init_nb_plotting)
from labtools.mplplots.init_nb_plotting import *

from labtools.mplplots import plots as mplplots
from labtools.mplplots import tools as mpltools

In [11]:
%run init.py

2018-03-31 04:51:31,384 : measurement : INFO : Logger set up!
C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:568: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Instrument sr1 already exists.
Instrument key1 already exists.
Instrument key2 already exists.
Instrument key3 already exists.
Instrument RF already exists.
Instrument yoko already exists.


Snapshot: Could not update parameter:current_range
Snapshot: Could not update parameter:current


In [13]:
from pytopo.data.data_storage import Data, GridData
Data.export_spyview = True
GridData.export_spyview = True

In [18]:
from pytopo.experiment.measurement import BaseMeasurement

class IVCurve(BaseMeasurement):
    
    start = 0.
    stop = +1.
    pts = 101
    nplc = 1
    delay = 0.001
        

    def measure(self):
        try:
            self.station.key1.NPLC(self.nplc)
            self.station.key2.NPLC(self.nplc)
        except (VisaIOError, AttributeError):
            self.station = check_keys(self.station.ivvi)
            self.station.key1.NPLC(self.nplc)
            self.station.key2.NPLC(self.nplc)
        
        vals = []
        bias = np.linspace(self.start, self.stop, self.pts)

        self.station.yoko.voltage(bias[0])
        time.sleep(1)

        t0 = time.time()
        
        for i, v in enumerate(bias):
            
            if not i%50:
                t1 = time.time() - t0
                print(f"Point {i}, t = {t1}s ...")
            
            self.station.yoko.voltage(v)
            time.sleep(self.delay)
            try:
                amp = self.station.key1.volt()
                volt = self.station.key2.volt()
            except (VisaIOError, AttributeError):
                self.station = check_keys(self.station.yoko)
                amp = self.station.key1.volt()
                volt = self.station.key2.volt()
            
            self.data.add(
            OrderedDict({
                'Bias' : {'value' : v * 1e-6, "unit": 'V', "independent_parameter": True},
                'Current' : {'value' : amp * 1e-7, "unit": 'A'},
                'Voltage' : {'value' : volt, 'unit' : 'V'},
                })
            )
            
            

In [19]:

t0 = time.time()
idx = 0
npts = 101

# while True:

#     for avgs in [201, 401, 801, 1601]:
    
    ### measure IV trace
m = IVCurve(station, namespace, info_string=f'{npts}_testing')
m.start = 0.
m.stop = 1.
m.pts = 101
m.nplc = 1
m.delay = 0.02
m.run()

bias = m.data['Current']['Bias [V]'].reshape(-1).copy()
i = m.data['Current']['Current [A]'].reshape(-1).copy()
v = m.data['Voltage']['Voltage [V]'].reshape(-1).copy()

i -= i[i.size//2]
bias *= 1e6
i *= 1e9

fig, ax = plt.subplots(1, 1)
ax.plot(bias, i, 'o', ms=1)
ax.set_xlabel('Bias voltage (uV)')
ax.set_ylabel('Current (nA)')
ax.set_title(m.datafilepath, size='x-small')
ax.grid(dashes=[2,2])
fig.savefig(m.data_prefix + ".png", dpi=300)

2018-03-31 05:01:24,466 : measurement : INFO : Ready to measure, file location: d:\data\2018-03\2018-03-31\2018-03-31_IVCurve-#0001_101_testing...
C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:568: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Point 0, t = 0.0s ...


AttributeError: 'Data' object has no attribute 'record_to_pages'

In [ ]:
station.yoko.vo